In [9]:
#All imports go here
#pip install shapely
#pip install spark-df-profiling

from shapely.geometry import shape
from shapely.ops import transform
import pyspark.sql.functions as pys

from pyspark.sql.types import IntegerType, DoubleType, StringType
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry import Point
from shapely import wkt

from pyspark.sql.functions import udf
import spark_df_profiling
      

In [2]:
#Read Fields data here
field = sqlContext.read.json("Box Sync/Nutrien_Jay_Share/checkIfFieldContainsSampleGeoLocation/fields/*.json")
field = field.select('field.boundary_map.boundary', 'field.id').withColumnRenamed('boundary','field_boundary')
field.show()

+--------------------+--------------------+
|      field_boundary|                  id|
+--------------------+--------------------+
|MULTIPOLYGON (((-...|7361a746-f379-11e...|
|MULTIPOLYGON (((-...|0e143638-241d-4f5...|
|MULTIPOLYGON (((-...|c5fdbde2-a4d0-447...|
|MULTIPOLYGON (((-...|a01838af-2607-11e...|
|MULTIPOLYGON (((-...|7efbd6d2-f379-11e...|
|MULTIPOLYGON (((-...|73ba46a1-f379-11e...|
|MULTIPOLYGON (((-...|5129bbd5-8a02-11e...|
+--------------------+--------------------+



In [3]:
#Read Fields data here
sample = sqlContext.read.json("Box Sync/Nutrien_Jay_Share/checkIfFieldContainsSampleGeoLocation/sample_event/*.txt")
sample = sample.withColumn('field_id', pys.explode('sample_event.associations.`agrian.field`'))
sample = sample.select('field_id', 'sample_event.centroid', 'sample_event.sample_event_type').withColumnRenamed('centroid','sample_centroid')
# sample = sample.withColumn('sample_centroid', sample.centroid )
sample.show()


+--------------------+--------------------+-----------------+
|            field_id|     sample_centroid|sample_event_type|
+--------------------+--------------------+-----------------+
|c5fdbde2-a4d0-447...|POINT (-94.075156...|             soil|
|7361a746-f379-11e...|POINT (-86.967842...|             soil|
|73ba46a1-f379-11e...|POINT (-89.338422...|             soil|
|0e143638-241d-4f5...|POINT (-93.188290...|             soil|
|7efbd6d2-f379-11e...|POINT (-77.490143...|             soil|
|5129bbd5-8a02-11e...|POINT (-82.891887...|             soil|
|a01838af-2607-11e...|POINT (-88.540064...|             soil|
+--------------------+--------------------+-----------------+



In [4]:
#Join the data on field_id
fieldAndSample = sample.join(field, sample.field_id == field.id).drop('id')
fieldAndSample.show()

+--------------------+--------------------+-----------------+--------------------+
|            field_id|     sample_centroid|sample_event_type|      field_boundary|
+--------------------+--------------------+-----------------+--------------------+
|c5fdbde2-a4d0-447...|POINT (-94.075156...|             soil|MULTIPOLYGON (((-...|
|7361a746-f379-11e...|POINT (-86.967842...|             soil|MULTIPOLYGON (((-...|
|73ba46a1-f379-11e...|POINT (-89.338422...|             soil|MULTIPOLYGON (((-...|
|0e143638-241d-4f5...|POINT (-93.188290...|             soil|MULTIPOLYGON (((-...|
|7efbd6d2-f379-11e...|POINT (-77.490143...|             soil|MULTIPOLYGON (((-...|
|5129bbd5-8a02-11e...|POINT (-82.891887...|             soil|MULTIPOLYGON (((-...|
|a01838af-2607-11e...|POINT (-88.540064...|             soil|MULTIPOLYGON (((-...|
+--------------------+--------------------+-----------------+--------------------+



In [5]:
def findIfPointInMultiPoly(point, MultiPoly):
    if MultiPoly == "" or point == "POINT EMPTY" or MultiPoly == "MULTIPOLYGON EMPTY" or point == "" or point == None or MultiPoly == None:
        return "Data missing"
    try:
        m = unicode(str(MultiPoly), "utf-8")
        m2 = wkt.loads(m)
        multiPoly = MultiPolygon(m2)
        
        p = unicode(str(point), "utf-8")
        p2 = wkt.loads(p)
        point = Point(p2)
        
        if multiPoly.contains(point):
            return "1"
        else:
            return "0"
    except:
        return "Error"

findIfPointInMultiPolyUdf = udf(lambda x, y: findIfPointInMultiPoly(x, y), StringType())



In [6]:
fieldAndSample.show()

+--------------------+--------------------+-----------------+--------------------+
|            field_id|     sample_centroid|sample_event_type|      field_boundary|
+--------------------+--------------------+-----------------+--------------------+
|c5fdbde2-a4d0-447...|POINT (-94.075156...|             soil|MULTIPOLYGON (((-...|
|7361a746-f379-11e...|POINT (-86.967842...|             soil|MULTIPOLYGON (((-...|
|73ba46a1-f379-11e...|POINT (-89.338422...|             soil|MULTIPOLYGON (((-...|
|0e143638-241d-4f5...|POINT (-93.188290...|             soil|MULTIPOLYGON (((-...|
|7efbd6d2-f379-11e...|POINT (-77.490143...|             soil|MULTIPOLYGON (((-...|
|5129bbd5-8a02-11e...|POINT (-82.891887...|             soil|MULTIPOLYGON (((-...|
|a01838af-2607-11e...|POINT (-88.540064...|             soil|MULTIPOLYGON (((-...|
+--------------------+--------------------+-----------------+--------------------+



In [7]:
fieldAndSample = fieldAndSample.select('*', findIfPointInMultiPolyUdf(fieldAndSample.sample_centroid, fieldAndSample.field_boundary).alias('sampleInField'))


In [8]:
fieldAndSample.show()

+--------------------+--------------------+-----------------+--------------------+-------------+
|            field_id|     sample_centroid|sample_event_type|      field_boundary|sampleInField|
+--------------------+--------------------+-----------------+--------------------+-------------+
|c5fdbde2-a4d0-447...|POINT (-94.075156...|             soil|MULTIPOLYGON (((-...|            1|
|7361a746-f379-11e...|POINT (-86.967842...|             soil|MULTIPOLYGON (((-...|            1|
|73ba46a1-f379-11e...|POINT (-89.338422...|             soil|MULTIPOLYGON (((-...|            1|
|0e143638-241d-4f5...|POINT (-93.188290...|             soil|MULTIPOLYGON (((-...|            1|
|7efbd6d2-f379-11e...|POINT (-77.490143...|             soil|MULTIPOLYGON (((-...|            1|
|5129bbd5-8a02-11e...|POINT (-82.891887...|             soil|MULTIPOLYGON (((-...|            1|
|a01838af-2607-11e...|POINT (-88.540064...|             soil|MULTIPOLYGON (((-...|            0|
+--------------------+--------

In [10]:
report = spark_df_profiling.ProfileReport(fieldAndSample)

In [11]:
report